In [ ]:
import pandas as pd

# read data in and show head
df = pd.read_csv('all_data.csv', encoding = 'latin1') #####需要改csv
df.head()
list(df)

In [ ]:
import sqlite3
import csv

conn = sqlite3.connect('nba.db') ####随便取个名字
c = conn.cursor()

In [ ]:
def create_database(conn):
    """
    Using the database connection conn, create tables appropriate for 
    storing and using IMDB data.  Drops any pre-exsiting tables that would be re-defined by this procedure.
    """
    c = conn.cursor()
    # YOUR CODE HERE
    c.execute('''DROP TABLE IF EXISTS games''')
    c.execute('''DROP TABLE IF EXISTS teams''')
    c.execute('''DROP TABLE IF EXISTS players''')
    c.execute('''DROP TABLE IF EXISTS fact_table''')
    
    c.execute('''
            CREATE TABLE games(
                id INTEGER,
                date text,
                home text,
                home_pts INTEGER,
                guest text,
                guest_pts INTEGER,
                game_duration INTEGER,
                PRIMARY KEY(id))
                ''')
    
    c.execute('''
            CREATE TABLE teams(
                id INTEGER,
                team text,
                GP INTEGER,
                W INTEGER,
                L INTEGER,
                WIN_perc float,
                MIN float,
                PTS float,
                FGM float,
                FGA float,
                FG_perc float,
                3PM
                PRIMARY KEY(id))
                ''')
    
    c.execute('''
            CREATE TABLE players(
                id INTEGER,
                team text,
                GP INTEGER,
                W INTEGER,
                L INTEGER,
                MIN float,
                PTS float,
                
                PRIMARY KEY(id))
                ''')
    
    c.execute('''
            CREATE TABLE fact_table(
                game_id INTEGER,
                team_id INTEGER,
                player_id INTEGER,
                FOREIGN KEY (game_id) REFERENCES games(id), 
                FOREIGN KEY (team_id) REFERENCES teams(id), 
                FOREIGN KEY (player_id) REFERENCES players(id)
                ''')
    conn.commit()
    
create_database(conn)

In [ ]:
def populate_database(conn, csv_file):    
    reader = csv.reader(open(csv_file, encoding='latin1'))
    game_dict=dict()
    team_dict=dict()
    player_dict=dict()
    game_id=0
    team_id=0
    player_id=0
    
    next(reader, None)
    for row in reader:
        row = [item.lower() for item in row]
        
        # insert into dimension tables
        
        #insert director table
        try:
            if row[0] not in director_dict:
                c.execute('''
                    INSERT INTO directors (director_name,director_facebook_likes) VALUES (?, ?)
                    ''', (row[0], row[3]))
                director_id+=1
                director_dict[row[0]]=director_id
        except sqlite3.Error as e:
            print("directors, insert error:", e.args[0])
        
        #insert actor1 table
        try:
            if row[9] not in actor_dict:
                c.execute('''
                    INSERT INTO actors (actor_name,actor_facebook_likes) VALUES (?, ?)
                    ''', (row[9], row[6]))
                actor_id+=1
                actor_dict[row[9]]=actor_id
        except sqlite3.Error as e:
            print("actor1, insert error:", e.args[0])
        
        #insert actor2 table
        try:
            if row[5] not in actor_dict:
                c.execute('''
                    INSERT INTO actors (actor_name,actor_facebook_likes) VALUES (?, ?)
                    ''', (row[5], row[21]))  
                actor_id+=1
                actor_dict[row[5]]=actor_id
        except sqlite3.Error as e:
            print("actor2, insert error:", e.args[0])
        
        #insert actor3 table
        try:
            if row[13] not in actor_dict:
                c.execute('''
                    INSERT INTO actors (actor_name,actor_facebook_likes) VALUES (?, ?)
                    ''', (row[13], row[4]))
                actor_id+=1
                actor_dict[row[13]]=actor_id
        except sqlite3.Error as e:
            print("actor3, insert error:", e.args[0])
        
        #insert movie table
        try:
            c.execute('''
                INSERT INTO movies (num_critic_for_reviews, duration, gross, genres, movie_title, num_voted_users,cast_total_facebook_likes, facenumber_in_poster, num_user_for_reviews, language, country, content_rating, budget, title_year, imdb_score, aspect_ratio, movie_facebook_likes) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''', (row[1], row[2], row[7],row[8], row[10], row[11], row[12], row[14], row[15], row[16], row[17], row[18], row[19], row[20], row[22], row[23], row[24]))       
        except sqlite3.Error as e:
            print("movies, insert error:", e.args[0])  
        
        
        # insert fact table
        # YOUR CODE HERE
        
        try:
            c.execute('''
                INSERT INTO fact_table (game_id, team_id, player_id) 
                SELECT directors.id ,a1.id, a2.id, a3.id, movies.id FROM directors, actors a1, actors a2, actors a3, movies
                WHERE directors.director_name = ? AND a1.actor_name = ? AND a2.actor_name = ? AND a3.actor_name = ? AND movies.movie_title = ?
                ''', (row[0], row[9], row[5], row[13], row[10]))
        except sqlite3.Error as e:
            print("fact_table, insert error:", e.args[0])

        conn.commit()
        
populate_database(conn, 'all_data.csv')

for t in['games','teams','players','fact_table']:
    print('%s:'%t)
    print(pd.read_sql_query("SELECT * FROM %s"%t, conn), '\n')